# Parcial de Regresión Lineal


## Preguntas de Negocio
1. **Predicción:** Si llega un carro con 92670.5 km, ¿a cuánto debería venderlo según el modelo?
2. **Inversión:** Como inversionista, ¿cuáles son los vehículos (según su kilometraje) que podrían generar un margen de ganancia atractivo?


In [4]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

%matplotlib inline

Como inversionista, los vehículos con un kilometraje relativamente bajo, pero aún dentro de un rango aceptable para su modelo y año, podrían generar un margen de ganancia atractivo.  Se buscan autos que, según el modelo predictivo, estén subvalorados en el mercado considerando su kilometraje.  Idealmente, se buscarían vehículos con un kilometraje que se encuentre por debajo del promedio del modelo, pero con un precio de venta que se sitúe por encima del precio predicho por el modelo, lo que indica una oportunidad de compra ventajosa.  A su vez, es importante considerar el estado general del vehículo, ya que factores como el mantenimiento y el historial de accidentes podrían afectar su precio de venta real.  Un análisis exhaustivo considerando el kilometraje en relación con el precio de venta estimado y otros factores relevantes (año, modelo, estado) permitirá identificar las mejores oportunidades de inversión.


## 1. Cargar y Preprocesar Datos

Se descarga el dataset de vehículos, se descomprime y se carga en un DataFrame. Además, se eliminan los valores faltantes.

In [5]:
# Descargar y descomprimir el dataset
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
!unzip -o cars_dataset.zip


--2025-03-12 20:23:43--  https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip [following]
--2025-03-12 20:23:43--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19753181 (19M) [application/zip]
Saving to: ‘cars_dataset.zip.1’

cars_dataset.zip.1  100%[===================>]  18.84M  87.3MB/s    in 0.2s    

2025-03-12 20:23:44 (87.3 MB/s) - ‘cars_dataset.zip.1’ saved [1975

In [13]:
df = pd.read_csv('car_prices.csv')
df.dropna(inplace=True)


## 2. Análisis Exploratorio de Datos (EDA)

Se generan estadísticas descriptivas para analizar las variables numéricas y entender la distribución de los datos.

In [15]:
# prompt: Se generan estadísticas descriptivas para analizar las variables numéricas y entender la distribución de los datos.

# Mostrar estadísticas descriptivas
print(df.describe())


                year      condition       odometer            mmr  \
count  472325.000000  472325.000000  472325.000000  472325.000000   
mean     2010.210980      30.774177   66701.732040   13837.058964   
std         3.822151      13.286866   51939.586894    9532.229273   
min      1990.000000       1.000000       1.000000      25.000000   
25%      2008.000000      24.000000   28137.000000    7425.000000   
50%      2012.000000      35.000000   51085.000000   12300.000000   
75%      2013.000000      41.000000   96590.000000   18300.000000   
max      2015.000000      49.000000  999999.000000  182000.000000   

        sellingprice  
count  472325.000000  
mean    13690.512058  
std      9613.033738  
min         1.000000  
25%      7200.000000  
50%     12200.000000  
75%     18200.000000  
max    230000.000000  


In [16]:
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


## 3. Modelo de Regresión Lineal

Se utiliza la variable 'odometer' para predecir 'sellingprice'. Se dividen los datos en conjunto de entrenamiento y prueba (80%-20%).

In [18]:
df.describe()

,year,condition,odometer,mmr,sellingprice
count,472325.000000,472325.000000,472325.000000,472325.000000,472325.000000
mean,2010.210980,30.774177,66701.732040,13837.058964,13690.512058
std,3.822151,13.286866,51939.586894,9532.229273,9613.033738
min,1990.000000,1.000000,1.000000,25.000000,1.000000
25%,2008.000000,24.000000,28137.000000,7425.000000,7200.000000
50%,2012.000000,35.000000,51085.000000,12300.000000,12200.000000
75%,2013.000000,41.000000,96590.000000,18300.000000,18200.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


In [22]:
X = df["odometer"].values.reshape(-1, 1)
y = df["sellingprice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")

Entrenamiento: 377860 muestras
Prueba: 94465 muestras


## 4. Evaluación del Modelo

Se evalúa el desempeño del modelo en el conjunto de prueba utilizando métricas como MSE, MAE y R².

In [23]:
# prompt: Se evalúa el desempeño del modelo en el conjunto de prueba utilizando métricas como MSE, MAE y R².

# Entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R^2: {r2}")


MSE: 60380220.39908442
MAE: 5399.032012605542
R^2: 0.3356777870212565


## 5. Predicción y Respuestas a Preguntas de Negocio

### Pregunta 1: Precio Estimado para un Carro con 92670.5 km

Utilizamos el modelo para predecir el precio de venta de un carro con 92670.5 km.

In [ ]:
# Predicción para un carro con 92670.5 km
#nuevo_odometro = np.array([[92670.5]])


In [25]:
# Crear instancia del modelo
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

LinearRegression()

In [26]:
# Extraer los coeficientes
intercept = model.intercept_
coef = model.coef_[0]

print(f"Intercept (beta_0): {intercept:.2f}")
print(f"Coeficiente (beta_1): {coef:.2f}")

Intercept (beta_0): 20836.52
Coeficiente (beta_1): -0.11


In [27]:
nuevo_odometro = np.array([[92670.5]])
precio_predicho = model.predict(nuevo_odometro)
print(f"Precio estimado para un carro con 92670.5 km: ${precio_predicho[0]:.2f}")


Precio estimado para un carro con 92670.5 km: $10916.68


## 6. Análisis Crítico de Resultados (10 Puntos Cerrados)

Responde a las siguientes preguntas de manera cerrada (Sí/No). Estas preguntas facilitan una calificación rápida:

1. ¿El coeficiente negativo (-0.107) indica que a mayor kilometraje, menor precio de venta? (Sí/No)
2. ¿Un R² de 0.336 sugiere que el modelo explica más del 50% de la variabilidad en el precio de venta? (Sí/No)
3. ¿El valor elevado del MSE indica que existen errores importantes en las predicciones? (Sí/No)
4. ¿El MAE obtenido es suficientemente bajo para garantizar predicciones precisas en todos los casos? (Sí/No)
5. ¿La eliminación de filas con valores faltantes puede reducir la robustez del modelo? (Sí/No)
6. ¿Incluir variables adicionales como año, condición y marca podría mejorar la capacidad predictiva del modelo? (Sí/No)
7. ¿La relación negativa entre kilometraje y precio es consistente con el comportamiento esperado en el mercado de vehículos? (Sí/No)
8. ¿Dividir los datos en 80% entrenamiento y 20% prueba es una práctica recomendada para evaluar el modelo? (Sí/No)



Sí. Un coeficiente negativo indica una relación inversa entre las variables.

No. Un R² de 0.336 significa que el modelo explica el 33.6% de la variabilidad, no más del 50%.

Sí. Un MSE elevado implica errores significativos en las predicciones del modelo.

No.  El MAE, aunque pueda ser relativamente bajo, no garantiza la precisión en todos los casos.  Siempre existirá un margen de error.

Sí. Eliminar filas con valores faltantes puede reducir la robustez del modelo al disminuir el tamaño de la muestra y potencialmente sesgar los datos.

Sí.  Incluir variables adicionales relevantes como año, condición y marca, usualmente mejora la precisión predictiva al capturar más información relevante.

Sí. Generalmente, el kilometraje mayor se asocia con un menor precio de venta en el mercado automotriz.

Sí. Dividir los datos en conjuntos de entrenamiento y prueba (como 80/20 o similar) es una práctica estándar para evaluar la capacidad de generalización del modelo.